In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time

In [2]:
df = pd.read_csv('X-IIoTID dataset_processed.csv')

In [3]:
df

,Scr_port,Des_port,Duration,Scr_bytes,Des_bytes,Conn_state,missed_bytes,Scr_pkts,Scr_ip_bytes,Des_pkts,...,combined_8.0,combined_9.0,combined_10.0,combined_11.0,combined_12.0,combined_13.0,combined_14.0,combined_15.0,combined_16.0,combined_17.0
0,0.751923,0.001223,7.266879e-05,0.008069,0.001804,1.0,0.0,0.001602,0.004507,0.001601,...,False,False,False,False,False,True,False,False,False,False
1,0.606818,0.000811,8.845092e-09,0.000047,0.000000,1.0,0.0,0.000031,0.000032,0.000000,...,False,False,True,False,False,False,False,False,False,False
2,0.901041,0.000811,1.413057e-08,0.000023,0.000002,1.0,0.0,0.000015,0.000016,0.000015,...,False,False,True,False,False,False,False,False,False,False
3,0.579305,0.028751,1.011628e-03,0.000673,0.000025,1.0,0.0,0.000122,0.000368,0.000091,...,False,False,True,False,False,False,False,False,False,False
4,0.770601,0.001223,5.559385e-04,0.000276,0.000035,1.0,0.0,0.000183,0.000372,0.000152,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596033,0.782656,0.001223,2.220237e-05,0.000076,0.000016,1.0,0.0,0.000092,0.000107,0.000076,...,False,False,False,False,False,False,False,False,False,False
596034,0.700574,0.028751,6.826952e-03,0.000000,0.000000,1.0,0.0,0.000061,0.000051,0.000046,...,False,False,True,False,False,False,False,False,False,False
596035,0.812565,0.001223,1.595088e-03,0.015668,0.031219,0.0,0.0,0.004104,0.008791,0.007854,...,False,False,True,False,False,False,False,False,False,False
596036,0.678264,0.295386,1.078670e-09,0.000000,0.000000,0.0,0.0,0.000015,0.000010,0.000015,...,False,False,False,False,False,False,False,False,False,False


In [4]:
X = (df.iloc[:,:-18]).drop(columns = ['class3'], axis=1)
# Y will be last 18 columns
y = df.iloc[:,-18:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional((LSTM(64))))
model.add(Dropout(0.1))
model.add((Dense(18, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='lstm_best_model.h5', verbose=1, save_best_only=True, monitor = 'val_accuracy', mode = 'max')
start = time.time()
# train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpointer])
end = time.time()
print(end - start)



Epoch 1/10


7451/7451 [==============================] - ETA: 0s - loss: 0.3220 - accuracy: 0.8992
Epoch 1: val_accuracy improved from -inf to 0.95333, saving model to lstm_best_model.h5
7451/7451 [==============================] - 2411s 314ms/step - loss: 0.3220 - accuracy: 0.8992 - val_loss: 0.1528 - val_accuracy: 0.9533
Epoch 2/10


C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7451/7451 [==============================] - ETA: 0s - loss: 0.1524 - accuracy: 0.9509
Epoch 2: val_accuracy improved from 0.95333 to 0.95611, saving model to lstm_best_model.h5
7451/7451 [==============================] - 1634s 219ms/step - loss: 0.1524 - accuracy: 0.9509 - val_loss: 0.1379 - val_accuracy: 0.9561
Epoch 3/10
7451/7451 [==============================] - ETA: 0s - loss: 0.1210 - accuracy: 0.9615
Epoch 3: val_accuracy improved from 0.95611 to 0.96629, saving model to lstm_best_model.h5
7451/7451 [==============================] - 1634s 219ms/step - loss: 0.1210 - accuracy: 0.9615 - val_loss: 0.1117 - val_accuracy: 0.9663
Epoch 4/10
7451/7451 [==============================] - ETA: 0s - loss: 0.1014 - accuracy: 0.9675
Epoch 4: val_accuracy improved from 0.96629 to 0.97377, saving model to lstm_best_model.h5
7451/7451 [==============================] - 1616s 217ms/step - loss: 0.1014 - accuracy: 0.9675 - val_loss: 0.0796 - val_accuracy: 0.9738
Epoch 5/10
7451/7451 [========

In [5]:
model = load_model('lstm_best_model.h5')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print("Time taken to predict: ", end-start)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

3726/3726 [==============================] - 129s 34ms/step
Time taken to predict:  131.3155927658081
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5168
           1       0.79      1.00      0.88       425
           2       1.00      0.96      0.98       299
           3       1.00      0.58      0.73      4011
           4       1.00      1.00      1.00      3280
           5       0.98      0.99      0.99       764
           6       0.99      0.99      0.99      6991
           7       1.00      1.00      1.00      3781
           8       1.00      0.12      0.21        17
           9       1.00      1.00      1.00       932
          10       0.97      1.00      0.98     67311
          11       1.00      1.00      1.00     15379
          12       1.00      0.83      0.90       167
          13       0.99      0.99      0.99      7633
          14       0.97      0.69      0.81       370
          15       0.00      0.00

C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334

In [6]:
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
for i in range(cm.shape[0]):
    print(i, cm[i][i])

Accuracy of each class:
0 1.0
1 0.9952941176470588
2 0.9632107023411371
3 0.5756669159810521
4 0.999390243902439
5 0.9882198952879581
6 0.9878415105135174
7 0.9978841576302565
8 0.11764705882352941
9 1.0
10 0.9977566816716436
11 0.9979192405227908
12 0.8263473053892215
13 0.9876850517489847
14 0.6918918918918919
15 0.0
16 0.0
17 0.9995926680244399
